In [1]:
import string, math
import numpy as np
from lxml import etree
from stemming.porter2 import stem
from collections import Counter


from tqdm.notebook import tqdm

In [ ]:
def tf_func (doc_id="query",text_list):
    tf_dict = {}
    x = Counter(text_list)
    max_occurence = x.most_common(1)[0][1]
    x = dict(sorted(x.items()))
    
    for word, count in x.items():
        tf = count/max_occurence
        tf_dict[doc_id].update({word,tf})
    return tf_dict

In [2]:
collectionName = "nytsmall"
#collectionName = "nyt199501"

tree = etree.parse( collectionName + ".xml")
root = tree.getroot()

In [3]:
vocab = {}
corpus = {}


for doc in tqdm(root.iter('DOC')):
    doc_txt = ""
    for element in doc.iter('HEADLINE', "P", "TEXT"):
        doc_txt += element.text
    doc_txt = doc_txt.translate(str.maketrans(
        '', '', string.punctuation)).lower()
    doc_txt = list(map(stem, doc_txt.split()))
    
    for word in set(doc_txt):
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1
            
    corpus[doc.attrib['id']] = doc_txt
vocab = dict(sorted(vocab.items()))

    

0it [00:00, ?it/s]

In [6]:
# sorting corpus dict
corpus = dict(sorted(corpus.items()))
n_docs = len(corpus)

idf_dict = {}

# IDF
with open (collectionName + '.idf', 'w') as file:
    for term, count in tqdm(vocab.items()):
        idf = math.log(n_docs/count)
        idf_dict[term] = idf
        file.write('{}\t{}\n'.format(term,idf))


  0%|          | 0/7736 [00:00<?, ?it/s]

In [4]:
# TF
tf_dict = {}
with open(collectionName+".tf","w") as file:
    for doc, text in tqdm(corpus.items()):
        x = Counter(text)
        max_occurence = x.most_common(1)[0][1]
        x = dict(sorted(x.items()))

        tf_dict[doc] = {}
        for word, count in x.items():
            tf = count/max_occurence
            tf_dict[doc].update({word:tf})
            file.write('{}\t{}\t{}\n'.format(doc,word,tf))

  0%|          | 0/102 [00:00<?, ?it/s]

In [7]:
tf_idf = {}

word_to_ix = {key:value for key,value in zip(vocab.keys(),range(len(vocab)))}

for doc_id,text in corpus.items():
    tf_idf[doc_id] = np.zeros(len(vocab))   
    
    for term in text:
        
        tf_idf[doc_id][word_to_ix[term]] = tf_dict[doc_id][term] *idf_dict[term]

In [11]:
def tf_func (doc_id="query",text_list):
    tf_dict = {}
    x = Counter(text)
    max_occurence = x.most_common(1)[0][1]
    x = dict(sorted(x.items()))
    
    for word, count in x.items():
        tf = count/max_occurence
        tf_dict[doc_id].update({word,tf})


31.597882657212608


In [ ]:
query = ['hurricane', 'philadelphia']
top10 = []

for doc_id, text in corpus.items():
    tf_idf_current = np.zeros(len(vocab))

    
    x = Counter(text)
    max_occurence = x.most_common(1)[0][1]
    x = dict(sorted(x.items()))
    
    for word,count in x.items():
        tf = count/max_occurence
    
    for term in text:
        tf_idf_current[word_to_ix[
            term]] = tf_dict[doc_id][term] * idf_dict[term]



# doc_txt = list(map(stem, doc_txt.split()))
# doc_txt = doc_txt.translate(str.maketrans('', '', string.punctuation))